In [3]:
# required libraries
import pandas as pd
import os
from pathlib import Path
# CHECK DATE 
import datetime
# parameters and URLs

#Gotta MAKE IT GLOBAL
path = str(Path(os.path.abspath(os.getcwd())).absolute())
csvurl=path+'/shopping_trends_updated.csv'
owl=path+'/cosumer_habits_ontology.rdf'
savePath =  path

# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD
TR = Namespace("http://www.semanticweb.org/aritz/ontologies/2023/10/costumerHabits#")



In [4]:
# Load the CSV files in memory
trends = pd.read_csv(csvurl, sep=',', index_col='Customer ID')
print(trends)

             Age  Gender Item Purchased     Category  Purchase Amount (USD)  \
Customer ID                                                                   
1             55    Male         Blouse     Clothing                     53   
2             19    Male        Sweater     Clothing                     64   
3             50    Male          Jeans     Clothing                     73   
4             21    Male        Sandals     Footwear                     90   
5             45    Male         Blouse     Clothing                     49   
...          ...     ...            ...          ...                    ...   
3896          40  Female         Hoodie     Clothing                     28   
3897          52  Female       Backpack  Accessories                     49   
3898          46  Female           Belt  Accessories                     33   
3899          44  Female          Shoes     Footwear                     77   
3900          52  Female        Handbag  Accessories

Person info

In [5]:
# Count the number of unique items in the "Item Purchased" column
unique_payment = trends['Payment Method'].unique()
unique_Category_count = trends['Category'].unique()

item_category_mapping = {}

# Iterate through the DataFrame and populate the dictionary
for index, row in trends.iterrows():
    item = row['Item Purchased']
    category = row['Category']
    size=row['Size']
    color=row['Color']
    location=row['Location']
    shippingtype=row['Shipping Type']
    
    # Check if the item is already in the dictionary, if not, add it with its category
    if item not in item_category_mapping:
        item_category_mapping[item] = category

print(unique_payment)


['Venmo' 'Cash' 'Credit Card' 'PayPal' 'Bank Transfer' 'Debit Card']


In [6]:

g = Graph()
g.parse(owl, format="xml")
g.bind("TR", TR)


# Iterate over the rows of the DataFrame
for index, row in trends.iterrows():
    # Create a URIRef for the movie based on the 'ID' column
    client_uri = URIRef(TR + "Client" + str(index))
    # Add triples using the movie_uri
    g.add((client_uri, RDF.type, TR.Customer))
    #fill the Client Class
    g.add((client_uri, TR['Age'], Literal(row['Age'], datatype=XSD.int)))
    g.add((client_uri, TR['Gender'], Literal(row['Gender'], datatype=XSD.string)))
    g.add((client_uri, TR['Review_Rating'], Literal(row['Review Rating'], datatype=XSD.string)))
    g.add((client_uri, TR['Discount_Applied'], Literal(row['Discount Applied'], datatype=XSD.string)))

    #fill the Habits Class
    habits_uri = URIRef(TR + "Habits" + str(index))
    g.add((habits_uri, TR['Previous_Purchases'], Literal(row['Previous Purchases'], datatype=XSD.int)))
    g.add((habits_uri, TR['Frequency_of_Purchases'], Literal(row['Frequency of Purchases'], datatype=XSD.string)))
    g.add((habits_uri, TR['Subscription_Status'], Literal(row['Subscription Status'], datatype=XSD.string)))
    
    #fill the Item Class
    item_uri = URIRef(TR + "Item" + str(index))
    g.add((item_uri, RDF.type, TR.Item))
    g.add((item_uri, TR['Item'], Literal(row['Item Purchased'],datatype=XSD.string)))
    g.add((item_uri, TR['Category'], Literal(row['Category'],datatype=XSD.string)))
    g.add((item_uri, TR['Size'], Literal(row['Size'],datatype=XSD.string)))
    g.add((item_uri, TR['Color'], Literal(row['Color'],datatype=XSD.string)))
    g.add((item_uri, TR['Location'], Literal(row['Location'],datatype=XSD.string)))
    g.add((item_uri, TR['Shipping_Method'], Literal(row['Shipping Type'],datatype=XSD.string)))
    g.add((item_uri, TR['Promo_Code_Used'], Literal(row['Promo Code Used'], datatype=XSD.string)))

    #fill the Payment Class
    Payment_uri = URIRef(TR + "Payment" + str(index))
    g.add((Payment_uri, RDF.type, TR.Payment))
    g.add((Payment_uri, TR['Payment_Method'], Literal(row['Payment Method'], datatype=XSD.string)))

    #fill the Price Class
    Price_uri = URIRef(TR + "Price" + str(index))
    g.add((Price_uri, RDF.type, TR.Price))
    g.add((Price_uri, TR['Price'], Literal(row['Purchase Amount (USD)'], datatype=XSD.float)))

    #fill the Season Class
    Season_uri = URIRef(TR + "Season" + str(index))
    g.add((Season_uri, RDF.type, TR.Season))
    g.add((Season_uri, TR['Season'], Literal(row['Season'], datatype=XSD.string)))

    #Create the relation
    #Client--Item
    g.add((client_uri, TR['hasBought'], item_uri))
    #ITEM--Price
    g.add((item_uri, TR['hasCosted'], Price_uri))
    #Customer--Habits
    g.add((client_uri, TR['hasHabits'], habits_uri))
    #Customer-Price
    g.add((client_uri, TR['payed'], Price_uri))
    #Price-Payment
    g.add((Price_uri, TR['payedBy'], Payment_uri))
    #Item--Season
    g.add((item_uri, TR['payedWhen'], Season_uri))
    #Habits--Payment
    g.add((habits_uri, TR['preferedMethod'], Payment_uri))



In [9]:
print(savePath)

c:\Users\PI.R\Desktop\Database\Project\Repo


In [8]:

print("--- saving serialization ---")
with open(savePath + 'Trends.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
